In [1]:
import os
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np 
import shapely 

import osm_flex.download as dl
import osm_flex.extract as ex
from osm_flex.config import OSM_DATA_DIR

from tqdm import tqdm

from lonboard import viz
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import Blues_9

from pathlib import Path
import pathlib

In [2]:
#define paths
p = Path('..')
data_path = Path(pathlib.Path.home().parts[0]) / 'Projects' / 'gmhcira' / 'data'
flood_data = Path(os.path.abspath(r'\\labsdfs.labs.vu.nl\labsdfs\BETA-IVM-BAZIS\eks510\fathom-global')) # 'Flood_data'
vul_data = data_path / 'Vulnerability'

### LOAD OSM DATA

In [3]:
iso3 = 'JAM' # Set the ISO3 country code
dl.get_country_geofabrik(iso3) # Use the download library to get the geofabrik data for the specified country

data_loc = OSM_DATA_DIR.joinpath('jamaica-latest.osm.pbf') # Specify the location of the OpenStreetMap (OSM) data file for Jamaica

assets = ex.extract_cis(data_loc, 'road') # Extract assets related to roads from the OSM data using the extraction module
assets = assets.loc[assets.geometry.geom_type == 'LineString'] # Filter assets to include only LineString geometries (roads)
roads = assets.rename(columns={'highway' : 'asset'}) # Rename the 'highway' column to 'asset' for better clarity

C:\Users\snn490\AppData\Local\miniforge3\envs\py311\Lib\site-packages\osgeo\ogr.py:560: FutureWarning: Neither ogr.UseExceptions() nor ogr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
extract points: 0it [00:00, ?it/s]
extract lines: 100%|███████████████████████████████████████████████████████████| 39974/39974 [00:10<00:00, 3772.22it/s]


In [4]:
viz(assets,width_min_pixels =1)

Map(layers=[PathLayer(table=pyarrow.Table
osm_id: string
highway: string
name: string
maxspeed: string
lanes: …

### LOAD HAZARD DATA

In [5]:
fluvial_data = flood_data / 'Jamaica' / 'fluvial_undefended'

In [6]:
fluvial_flood = list(fluvial_data.iterdir())[2] # get second item of list with data layers

In [7]:
flood_map = xr.open_dataset(fluvial_flood, engine="rasterio")

In [8]:
flood_map_vector = flood_map['band_data'].to_dataframe().reset_index() #transform to dataframe

#remove data that will not be used
flood_map_vector = flood_map_vector.loc[(flood_map_vector.band_data > 0) & (flood_map_vector.band_data < 100)]

# create geometry values and drop lat lon columns
flood_map_vector['geometry'] = [shapely.points(x) for x in list(zip(flood_map_vector['x'],flood_map_vector['y']))]
flood_map_vector = flood_map_vector.drop(['x','y','band','spatial_ref'],axis=1)

# drop all non values to reduce size
flood_map_vector = flood_map_vector.loc[~flood_map_vector['band_data'].isna()].reset_index(drop=True)

# and turn them into squares again:
flood_map_vector.geometry= shapely.buffer(flood_map_vector.geometry,distance=0.00083/2,cap_style='square').values

In [9]:
viz(gpd.GeoDataFrame(flood_map_vector.copy()),
   get_fill_color = apply_continuous_cmap(flood_map_vector.band_data, Blues_9))

Map(layers=[SolidPolygonLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x000001ADACF495A0>
[
 …

### LOAD VULNERABILITY DATA

In [10]:
curves = pd.read_excel(vul_data / 'Table_D2_Multi-Hazard_Fragility_and_Vulnerability_Curves_V1.0.0.xlsx',sheet_name = 'F_Vuln_Depth',index_col=[0],header=[0,1,2,3,4])
infra_curves = curves.loc[:, curves.columns.get_level_values('Infrastructure description').str.contains('Roads')]

In [11]:
maxdam = pd.read_excel(vul_data / 'Table_D3_Costs_V1.0.0.xlsx',sheet_name='Cost_Database',index_col=[0])

### PERFORM DAMAGE ASSESSMENT

In [12]:
def overlay_hazard_assets(df_ds,assets):
    """
    Overlay hazard assets on a dataframe of spatial geometries.
    Arguments:
        *df_ds*: GeoDataFrame containing the spatial geometries of the hazard data. 
        *assets*: GeoDataFrame containing the infrastructure assets.
    Returns:
        *geopandas.GeoSeries*: A GeoSeries containing the spatial geometries of df_ds that intersect with the infrastructure assets.
    """
    #overlay 
    hazard_tree = shapely.STRtree(df_ds.geometry.values)
    if (shapely.get_type_id(assets.iloc[0].geometry) == 3) | (shapely.get_type_id(assets.iloc[0].geometry) == 6): # id types 3 and 6 stand for polygon and multipolygon
        return  hazard_tree.query(assets.geometry,predicate='intersects')    
    else:
        return  hazard_tree.query(assets.buffered,predicate='intersects')

def buffer_assets(assets,buffer_size=0.00083):
    """
    Buffer spatial assets in a GeoDataFrame.
    Arguments:
        *assets*: GeoDataFrame containing spatial geometries to be buffered.
        *buffer_size* (float, optional): The distance by which to buffer the geometries. Default is 0.00083.
    Returns:
        *GeoDataFrame*: A new GeoDataFrame with an additional 'buffered' column containing the buffered geometries.
    """
    assets['buffered'] = shapely.buffer(assets.geometry.values,distance=buffer_size)
    return assets

def get_damage_per_asset(asset,flood_numpified,asset_geom,asset_type,curve,maxdam):
    """
    Calculate damage for a given asset based on hazard information.
    Arguments:
        *asset*: Tuple containing information about the asset. It includes:
            - Index or identifier of the asset (asset[0]).
            - Asset-specific information, including hazard points (asset[1]['hazard_point']).
        *flood_numpified*: NumPy array representing flood hazard information.
        *asset_geom*: Shapely geometry representing the spatial coordinates of the asset.
        *asset_type*: String representing type of asset.
        *curve*: Pandas DataFrame representing the curve for the asset type.
        *maxdam*: Maximum damage value.
    Returns:
        *tuple*: A tuple containing the asset index or identifier and the calculated damage.
    """
    
    # find the exact hazard overlays:
    get_hazard_points = flood_numpified[asset[1]['hazard_point'].values] 
    get_hazard_points[shapely.intersects(get_hazard_points[:,1],asset_geom)]

    # get maxdam and curves
    maxdam_asset = maxdam
    hazard_intensity = curve.index.values
    fragility_values = curve.iloc[:,0].values

    # estimate damage
    if len(get_hazard_points) == 0: # no overlay of asset with hazard
        return asset[0],0
    else:
        overlay_meters = shapely.length(shapely.intersection(get_hazard_points[:,1],asset_geom)) # get the length of exposed meters per hazard cell
        return asset[0],np.sum((np.interp(np.float16(get_hazard_points[:,0]),hazard_intensity,fragility_values))*overlay_meters*maxdam_asset) #return asset number, total damage for asset number (damage factor * meters * max. damage)

In [13]:
%%time
overlay_roads = pd.DataFrame(overlay_hazard_assets(flood_map_vector,buffer_assets(roads)).T,columns=['asset','hazard_point'])

CPU times: total: 11.1 s
Wall time: 11.4 s


In [14]:
maxdams = maxdam.loc['Roads','Amount'].dropna()[:10]

In [15]:
collect_output = {}
geom_dict = roads['geometry'].to_dict()
type_dict = roads['asset'].to_dict()

flood_numpified = flood_map_vector.to_numpy() # convert dataframe to numpy array
for infra_curve in infra_curves:
    curve = infra_curves[infra_curve[0]]
    for maxdam in maxdams:
        for asset in tqdm(overlay_roads.groupby('asset'),total=len(overlay_roads.asset.unique()), #group asset items for different hazard points per asset and get total number of unique assets
                              desc=f'polyline damage calculation for {infra_curve[0]} with a max dam of {maxdam}'):
            asset_geom = geom_dict[asset[0]]
            asset_type = type_dict[asset[0]]
            collect_output[infra_curve[0],maxdam] = get_damage_per_asset(asset,flood_numpified,asset_geom,asset_type,curve,maxdam)

polyline damage calculation for F7.1 with a max dam of 909.3454827565133: 100%|███| 3243/3243 [00:03<00:00, 979.71it/s]
polyline damage calculation for F7.1 with a max dam of 909.3454827565133: 100%|██| 3243/3243 [00:03<00:00, 1041.56it/s]
polyline damage calculation for F7.1 with a max dam of 7.7971745573977564: 100%|█| 3243/3243 [00:02<00:00, 1128.40it/s]
polyline damage calculation for F7.1 with a max dam of 5746.447358140652: 100%|███| 3243/3243 [00:03<00:00, 960.62it/s]
polyline damage calculation for F7.1 with a max dam of 1340.8377168994855: 100%|█| 3243/3243 [00:02<00:00, 1207.03it/s]
polyline damage calculation for F7.1 with a max dam of 660.841446186175: 100%|███| 3243/3243 [00:02<00:00, 1173.03it/s]
polyline damage calculation for F7.1 with a max dam of 95.77412263567753: 100%|███| 3243/3243 [00:03<00:00, 893.56it/s]
polyline damage calculation for F7.1 with a max dam of 114.92894716281305: 100%|██| 3243/3243 [00:03<00:00, 910.19it/s]
polyline damage calculation for F7.1 wit